In [0]:
df = spark.read.table("main_prod.ml_data.static_moving_worktype")
display(df)

In [0]:
df_static = df.where("predicted_work_type = 'moving'")
display(df_static)

In [0]:
# in df_static, count unique userid for each employerid
from pyspark.sql.functions import countDistinct
df_static_count = df_static.groupBy("employername").agg(countDistinct("userid").alias("count")).orderBy("count", ascending=False)
display(df_static_count)

In [0]:
# in df, count unique userid for each employerid for both static and moving

from pyspark.sql.functions import countDistinct
df_count = df.where().groupBy("employername").agg(countDistinct("userid").alias("count"))


In [0]:
import pyspark.sql.functions as F
df = spark.read.table("main_prod.ml_data.static_moving_worktype")
df_v2 = df.withColumn("is_static", F.when(F.col("predicted_work_type") == "static", 1).otherwise(0))
df_v3 = df_v2.select('userid', 'employername','is_static').distinct()

display(df_v3)


In [0]:
# sum is_static and count userid for each employername
from pyspark.sql.functions import sum
df_v3.createOrReplaceTempView("df_v3")
# df_v5 = df_v4.withColumn("moving", (F.col("total_count") - F.col("static_count"))/F.col("total_count"))
# display(df_v4)
query = """
select employername, count(userid) as total_count, sum(is_static) as static_count from df_v3 group by employername
"""
df_v4 = spark.sql(query)
display(df_v4)
# df_v4 = df_v3.groupBy("employername")

In [0]:
df_v4 = df_v4.withColumn('moving_count', F.col("total_count") - F.col("static_count"))
df_v5 = df_v4.withColumn("moving_per", F.col("moving_count")/F.col("total_count")).orderBy("moving_count", "moving_per", ascending=False)
display(df_v5)

In [0]:
# take the latest worktype of each user
df = spark.read.table("main_prod.ml_data.static_moving_worktype")
display(df)

In [0]:
df = spark.read.parquet("/Volumes/main_prod/datascience_scratchpad/jatin/trajcl_exp/usa/backfill_static_moving_relax_condition_v2")
from pyspark.sql.window import Window
import pyspark.sql.functions as F

window_spec = Window.partitionBy("userid").orderBy(F.col("paydate").desc())
df_latest = df.withColumn("rn", F.row_number().over(window_spec)).filter(F.col("rn") == 1).select("userid", "predicted_work_type", "employername")
display(df_latest.toPandas())

In [0]:
%sql
select count(distinct(userid)) from main_prod.earnings_analysis.fact_user_earnings_daily


In [0]:
df_latest.where('predicted_work_type = "moving"').count()/df_latest.count()

In [0]:
df_latest.where('predicted_work_type = "moving"').count()

In [0]:
df_latest.count()

In [0]:
import pyspark.sql.functions as F

df_v2 = df_latest.withColumn("is_static", F.when(F.col("predicted_work_type") == "static", 1).otherwise(0))
df_v3 = df_v2.select('userid', 'employername','is_static').distinct()

display(df_v3)

In [0]:
from pyspark.sql.functions import sum
df_v3.createOrReplaceTempView("df_v3")
# df_v5 = df_v4.withColumn("moving", (F.col("total_count") - F.col("static_count"))/F.col("total_count"))
# display(df_v4)
query = """
select employername, count(userid) as total_count, sum(is_static) as static_count from df_v3 group by employername
"""
df_v4 = spark.sql(query)
display(df_v4)

In [0]:
df_v4 = df_v4.withColumn('moving_count', F.col("total_count") - F.col("static_count"))
df_v5 = df_v4.withColumn("moving_per", F.col("moving_count")/F.col("total_count")).orderBy("total_count", "moving_per", ascending=False)
display(df_v5)

In [0]:
display(df_v5.where('moving_per<0.05'))

In [0]:
display(df_v5.where('moving_per>0.7'))